# Sprint1課題　機械学習フロー

1.この課題の目的
機械学習の実践的な流れを知る
高い汎化性能を持つモデルを完成させる

2.機械学習フロー
1回目のSprintでは、機械学習の実践的な流れを抑えます。
このSprintはWeek3,4の延長になり、引き続きKaggleの Home Credit Default Risk コンペティションを用います。
特にここでは適切な 検証 を組み込むことを重視し、その上で前処理やモデルの選定を進めていきます。

適切な検証を行い、テストデータに対して高い汎化性能を持つモデルを完成させましょう。

In [1]:
import pandas as pd
import numpy as npa
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
pd.options.display.max_columns = None

df_train=pd.read_csv('./application_train.csv')
df_test_X=pd.read_csv('./application_test.csv')

In [2]:
# 文字列をダミー変数に置き換える        
df_train = pd.get_dummies(df_train)
df_test_X = pd.get_dummies(df_test_X)

In [3]:
# flaot16に揃えるとintがinfになる謎。解明は未来に持ち越す
# df_train = df_train.astype('float16')
# df_test_X = df_test_X.astype('float16')

In [4]:
# 欠損値がないIndexのみ残す
df_train = df_train.dropna(thresh=121)
df_train = df_train.dropna(how='any').dropna(how='any', axis=1)

In [5]:
df_train.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR,NAME_CONTRACT_TYPE_Cash loans,NAME_CONTRACT_TYPE_Revolving loans,CODE_GENDER_F,CODE_GENDER_M,CODE_GENDER_XNA,FLAG_OWN_CAR_N,FLAG_OWN_CAR_Y,FLAG_OWN_REALTY_N,FLAG_OWN_REALTY_Y,NAME_TYPE_SUITE_Children,NAME_TYPE_SUITE_Family,NAME_TYPE_SUITE_Group of people,NAME_TYPE_SUITE_Other_A,NAME_TYPE_SUITE_Other_B,"NAME_TYPE_SUITE_Spouse, partner",NAME_TYPE_SUITE_Unaccompanied,NAME_INCOME_TYPE_Businessman,NAME_INCOME_TYPE_Commercial associate,NAME_INCOME_TYPE_Maternity leave,NAME_INCOME_TYPE_Pensioner,NAME_INCOME_TYPE_State servant,NAME_INCOME_TYPE_Student,NAME_INCOME_TYPE_Unemployed,NAME_INCOME_TYPE_Working,NAME_EDUCATION_TYPE_Academic degree,NAME_EDUCATION_TYPE_Higher education,NAME_EDUCATION_TYPE_Incomplete higher,NAME_EDUCATION_TYPE_Lower secondary,NAME_EDUCATION_TYPE_Secondary / secondary special,NAME_FAMILY_STATUS_Civil marriage,NAME_FAMILY_STATUS_Married,NAME_FAMILY_STATUS_Separated,NAME_FAMILY_STATUS_Single / not married,NAME_FAMILY_STATUS_Unknown,NAME_FAMILY_STATUS_Widow,NAME_HOUSING_TYPE_Co-op apartment,NAME_HOUSING_TYPE_House / apartment,NAME_HOUSING_TYPE_Municipal apartment,NAME_HOUSING_TYPE_Office apartment,NAME_HOUSING_TYPE_Rented apartment,NAME_HOUSING_TYPE_With parents,OCCUPATION_TYPE_Accountants,OCCUPATION_TYPE_Cleaning staff,OCCUPATION_TYPE_Cooking staff,OCCUPATION_TYPE_Core staff,OCCUPATION_TYPE_Drivers,OCCUPATION_TYPE_HR staff,OCCUPATION_TYPE_High skill tech staff,OCCUPATION_TYPE_IT staff,OCCUPATION_TYPE_Laborers,OCCUPATION_TYPE_Low-skill Laborers,OCCUPATION_TYPE_Managers,OCCUPATION_TYPE_Medicine staff,OCCUPATION_TYPE_Private service staff,OCCUPATION_TYPE_Realty agents,OCCUPATION_TYPE_Sales staff,OCCUPATION_TYPE_Secretaries,OCCUPATION_TYPE_Security staff,OCCUPATION_TYPE_Waiters/barmen staff,WEEKDAY_APPR_PROCESS_START_FRIDAY,WEEKDAY_APPR_PROCESS_START_MONDAY,WEEKDAY_APPR_PROCESS_START_SATURDAY,WEEKDAY_APPR_PROCESS_START_SUNDAY,WEEKDAY_APPR_PROCESS_START_THURSDAY,WEEKDAY_APPR_PROCESS_START_TUESDAY,WEEKDAY_APPR_PROCESS_START_WEDNESDAY,ORGANIZATION_TYPE_Advertising,ORGANIZATION_TYPE_Agriculture,ORGANIZATION_TYPE_Bank,ORGANIZATION_TYPE_Business

In [6]:
df_train["SK_ID_CURR"].shape

(11351,)

In [7]:
type(df_train["SK_ID_CURR"])

pandas.core.series.Series

In [8]:
df_train["SK_ID_CURR"]

71        100083
124       100145
143       100165
152       100179
161       100190
164       100193
249       100289
255       100295
296       100341
298       100343
316       100363
319       100366
323       100371
328       100376
337       100389
343       100395
369       100425
415       100479
441       100508
483       100557
492       100567
580       100666
591       100678
626       100716
696       100805
701       100810
739       100852
765       100879
803       100920
862       100988
           ...  
306803    455460
306814    455471
306882    455551
306894    455563
306908    455578
306911    455581
306956    455628
306957    455629
306958    455630
306967    455642
306969    455646
307007    455687
307039    455723
307040    455724
307095    455785
307149    455847
307172    455875
307174    455877
307199    455904
307206    455914
307215    455923
307242    455951
307259    455976
307319    456043
307358    456083
307359    456084
307363    456088
307407    4561

In [9]:
# 変数を定義する
df_train_y = df_train['TARGET']
df_train_X = df_train #del ...['TARGET']するために定義
del df_train_X['TARGET']
df_X = pd.concat([df_train_X, df_test_X])

### 【問題1】クロスバリデーション

事前学習期間は検証用データを分割しておき、それに対して指標値を計算することで検証を行っていました。
しかし、分割の仕方により精度は変化します。実践的には クロスバリデーション を行います。

具体的には分割を複数回行い、それぞれに対して学習と検証を行う方法です。
複数回の分割を行う関数はscikit-learnにKFoldとして用意されています。

sklearn.model_selection.KFold — scikit-learn 0.20.2 documentation

In [10]:
#　欠損値の数を確認する
print(df_train_X.isnull().sum())

SK_ID_CURR                                  0
CNT_CHILDREN                                0
AMT_INCOME_TOTAL                            0
AMT_CREDIT                                  0
AMT_ANNUITY                                 0
AMT_GOODS_PRICE                             0
REGION_POPULATION_RELATIVE                  0
DAYS_BIRTH                                  0
DAYS_EMPLOYED                               0
DAYS_REGISTRATION                           0
DAYS_ID_PUBLISH                             0
OWN_CAR_AGE                                 0
FLAG_MOBIL                                  0
FLAG_EMP_PHONE                              0
FLAG_WORK_PHONE                             0
FLAG_CONT_MOBILE                            0
FLAG_PHONE                                  0
FLAG_EMAIL                                  0
CNT_FAM_MEMBERS                             0
REGION_RATING_CLIENT                        0
REGION_RATING_CLIENT_W_CITY                 0
HOUR_APPR_PROCESS_START           

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [12]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5, shuffle=True)
kf.get_n_splits(df_train)
print(kf.get_n_splits(df_train))

5


In [13]:
# DFをarrayに変換（DFだとkf.splitが動かないので）
ar_train_X = df_train_X.values
ar_train = df_train.values
ar_train_y = df_train_y.values

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
result=[]

for train_index, test_index in kf.split(ar_train):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = ar_train[train_index], ar_train[test_index]
    y_train, y_test = ar_train_y[train_index], ar_train_y[test_index]
    clf = RandomForestClassifier()    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test) 
  
    auc = roc_auc_score(y_test, y_pred)
    
    result.append(auc)
    
result = np.array(result)
print(result)
print("AUC: {:.2f}+/-{:.2f}".format(result.mean(),result.std()))


TRAIN: [    0     1     2 ..., 11348 11349 11350] TEST: [    5     7    13 ..., 11341 11342 11344]
TRAIN: [    0     1     2 ..., 11348 11349 11350] TEST: [    3     9    18 ..., 11313 11333 11337]
TRAIN: [    3     5     6 ..., 11348 11349 11350] TEST: [    0     1     2 ..., 11338 11340 11343]
TRAIN: [    0     1     2 ..., 11347 11349 11350] TEST: [    6    19    20 ..., 11336 11339 11348]
TRAIN: [    0     1     2 ..., 11343 11344 11348] TEST: [    8    10    11 ..., 11347 11349 11350]
[ 0.50652308  0.49976393  0.50396825  0.49953271  0.50373134]
AUC: 0.50+/-0.00


### 【問題2】グリッドサーチ

これまで分類器のパラメータは基本的にデフォルトの設定を使用していました。
パラメータの詳細は今後のSprintで学んでいくことになりますが、パラメータは状況に応じて最適なものを選ぶ必要があります。
パラメータを探索するために グリッドサーチ と呼ばれる総当たり的手法が一般的に利用されます。
グリッドサーチをパイプラインの中に組み込みましょう。

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

# 平均二乗誤差を評価するためのメソッドを呼び出し
from sklearn.metrics import mean_squared_error

# モデル
svc = SVC()

# 試行するパラメータを羅列する
param_grid = [
    {'C': [1, 10], 'kernel': ['linear']},
#     {'C': [1, 10, 100, 1000], 'kernel': ['rbf'], 'gamma': [0.001, 0.0001]},
#     {'C': [1, 10, 100, 1000], 'kernel': ['poly'], 'degree': [2, 3, 4], 'gamma': [0.001, 0.0001]},
#     {'C': [1, 10, 100, 1000], 'kernel': ['sigmoid'], 'gamma': [0.001, 0.0001]}
    ]

# SVC(分類器)を渡す、試行してほしいパラメータを渡す、4-Fold CV で汎化性能を調べる 3つの条件でgscというインスタンス化
gsc = GridSearchCV(svc, param_grid,cv=4) 

# グリッドサーチで優れたハイパーパラメータを探す
gsc.fit(ar_train_X, ar_train_y)  

print(gsc.best_score_)  # 最も良かったスコア
print(gsc.best_params_)  # 上記を記録したパラメータの組み合わせ


In [ ]:
y_pred_best = gsc.best_estimator_.predict_proba(X_test)
y_pred_best

### 【問題3】Kernelからの調査

In [ ]:
KaggleのKernelから自身にはなかったアイデアを見つけ出して、列挙してください。
そして、効果があると考えられるものを検証してください。

### 【問題4】高い汎化性能のモデル

In [ ]:
これまで学んだことを用いながら汎化性能の高いモデルを作成してください。
今は全体の流れを掴むことを重視し、Sprintの時間内に結果を出すということも意識しましょう。